In [58]:
import base64
import csv
from datetime import datetime
import json
import re

import pandas as pd

# Read in HAR file as JSON
with open('mini_2021_2016.har', 'r', encoding='utf8') as f:
    raw = json.loads(f.read())

# Filter for entries
entries = raw['log']['entries']

# Match pattern for relevant URLs
pattern = re.compile("https:\/\/www\.instagram\.com\/graphql\/query\/\?query_hash=.*")

# Filter for relevant request urls
url_indices = []
for i in range(len(entries)):
    if pattern.match(entries[i]['request']['url']):
        url_indices.append(i)
    else:
        continue

print(f"Number of relevant request urls: {len(url_indices)}")
print(f"Indices of relevant URL entries: {url_indices}")

Number of relevant request urls: 220
Indices of relevant URL entries: [16, 24, 99, 115, 128, 141, 157, 170, 186, 199, 215, 228, 241, 257, 270, 283, 299, 312, 325, 338, 354, 367, 380, 394, 407, 420, 436, 449, 462, 478, 491, 504, 520, 533, 546, 562, 573, 586, 602, 615, 626, 642, 655, 666, 682, 695, 711, 724, 737, 751, 764, 777, 793, 806, 819, 835, 847, 860, 876, 889, 902, 918, 931, 944, 960, 973, 984, 997, 1013, 1026, 1039, 1052, 1068, 1081, 1094, 1107, 1120, 1136, 1149, 1165, 1178, 1191, 1204, 1220, 1239, 1252, 1268, 1281, 1294, 1307, 1323, 1336, 1349, 1366, 1379, 1392, 1405, 1418, 1431, 1447, 1460, 1473, 1489, 1502, 1515, 1531, 1544, 1557, 1570, 1586, 1599, 1612, 1625, 1641, 1654, 1667, 1683, 1696, 1709, 1722, 1738, 1751, 1764, 1777, 1793, 1806, 1819, 1832, 1848, 1861, 1874, 1887, 1903, 1916, 1929, 1942, 1958, 1971, 1984, 1997, 2013, 2026, 2039, 2055, 2068, 2081, 2094, 2110, 2123, 2136, 2152, 2165, 2178, 2191, 2207, 2220, 2233, 2246, 2262, 2275, 2288, 2301, 2314, 2330, 2343, 2356, 2369

In [59]:
# Extracting relevant text from utf8 and base64 encoded content
valid_posts = []
browsed_at = []

for index in url_indices:
    try:
        # utf8: Load JSON
        post = json.loads(entries[index]['response']['content']['text'])
        valid_posts.append(post)
        browsed = datetime.strptime(entries[index]['startedDateTime'], "%Y-%m-%dT%H:%M:%S.%fZ")
        browsed_at.append(browsed)
        
    except ValueError: # Raised if encoded in base64
        try:
            # Decode base64: Load JSON
            post = json.loads(base64.b64decode(entries[index]['response']['content']['text']))
            valid_posts.append(post)
            browsed = datetime.strptime(entries[index]['startedDateTime'], "%Y-%m-%dT%H:%M:%S.%fZ")
            browsed_at.append(browsed)
                             
        except (ValueError, KeyError) as error:
            continue
        
    except KeyError:
        continue

print(f"valid_posts: {len(valid_posts)}")
print(f"browsed: {len(browsed_at)}")

valid_posts: 126
browsed: 126


In [60]:
# Create dataframe to hold relevant data
df_posts = pd.DataFrame({'medium_id': pd.Series([], dtype='int64'),
                         'updated': pd.Series([], dtype='datetime64[ns]'),
                         'shortcode': pd.Series([], dtype='str'), 
                         'owner_id': pd.Series([], dtype='int64'),
                         'username': pd.Series([], dtype='str'),
                         'taken_at': pd.Series([], dtype='datetime64[ns]'),
                         'typename': pd.Series([], dtype='int64'),
                         'is_video': pd.Series([], dtype='bool'),
                         'caption': pd.Series([], dtype='str'),
                         'likes': pd.Series([], dtype='int64'),
                         'comments': pd.Series([], dtype='int64'),
                         'video_views': pd.Series([], dtype='int64')})

df_posts.set_index(['medium_id'])

# Extract data from posts
for i in range(len(valid_posts)):
    try:
        # Works with single posts
        medium_id = valid_posts[i]['data']['shortcode_media']['id']
        updated = browsed_at[i]
        shortcode = valid_posts[i]['data']['shortcode_media']['shortcode']
        owner_id = valid_posts[i]['data']['shortcode_media']['owner']['id']
        username = valid_posts[i]['data']['shortcode_media']['owner']['username']
        taken_at = datetime.fromtimestamp(valid_posts[i]['data']['shortcode_media']['taken_at_timestamp'])
        typename = valid_posts[i]['data']['shortcode_media']['__typename']
        is_video = valid_posts[i]['data']['shortcode_media']['is_video']
        caption = valid_posts[i]['data']['shortcode_media']['edge_media_to_caption']['edges'][0]['node']['text']
        likes = valid_posts[i]['data']['shortcode_media']['edge_media_preview_like']['count']
        comments = valid_posts[i]['data']['shortcode_media']['edge_media_to_parent_comment']['count']
        if is_video:
            video_views = valid_posts[i]['data']['shortcode_media']['video_view_count']
        else:
            video_views = None
        print(f"parsed single post {i}: {shortcode}")
        df_posts = df_posts.append({'medium_id': medium_id, 'updated': updated, 'shortcode': shortcode, 'owner_id': owner_id, 'username': username, 'taken_at': taken_at, 'typename': typename, 'is_video': is_video, 'caption': caption, 'likes': likes, 'comments': comments, 'video_views': video_views}, ignore_index=True)
    
    except KeyError: # Raised with timeline
        try:
            # Works with timeline
            for j in range(len(valid_posts[i]['data']['user']['edge_owner_to_timeline_media']['edges'])):
                try:
                    medium_id = valid_posts[i]['data']['user']['edge_owner_to_timeline_media']['edges'][j]['node']['id']
                    updated = browsed_at[i]
                    shortcode = valid_posts[i]['data']['user']['edge_owner_to_timeline_media']['edges'][j]['node']['shortcode']
                    owner_id = valid_posts[i]['data']['user']['edge_owner_to_timeline_media']['edges'][j]['node']['owner']['id']
                    username = valid_posts[i]['data']['user']['edge_owner_to_timeline_media']['edges'][j]['node']['owner']['username']
                    taken_at = datetime.fromtimestamp(valid_posts[i]['data']['user']['edge_owner_to_timeline_media']['edges'][j]['node']['taken_at_timestamp'])
                    typename = valid_posts[i]['data']['user']['edge_owner_to_timeline_media']['edges'][j]['node']['__typename']
                    is_video = valid_posts[i]['data']['user']['edge_owner_to_timeline_media']['edges'][j]['node']['is_video']
                    caption = valid_posts[i]['data']['user']['edge_owner_to_timeline_media']['edges'][j]['node']['edge_media_to_caption']['edges'][0]['node']['text']
                    likes = valid_posts[i]['data']['user']['edge_owner_to_timeline_media']['edges'][j]['node']['edge_media_preview_like']['count']
                    comments = valid_posts[i]['data']['user']['edge_owner_to_timeline_media']['edges'][j]['node']['edge_media_to_comment']['count']
                    if is_video:
                        video_views = valid_posts[i]['data']['user']['edge_owner_to_timeline_media']['edges'][j]['node']['video_view_count']
                    else:
                        video_views = None
                    print(f"parsed timeline post {j} of entry {i}: {shortcode}")
                    df_posts = df_posts.append({'medium_id': medium_id, 'updated': updated, 'shortcode': shortcode, 'owner_id': owner_id, 'username': username, 'taken_at': taken_at, 'typename': typename, 'is_video': is_video, 'caption': caption, 'likes': likes, 'comments': comments, 'video_views': video_views}, ignore_index=True)
        
                except KeyError:
                    print("Continued KeyError")
        
        except KeyError:
            print(f"KeyError at {i}:{valid_posts[i]}")
df_posts

parsed timeline post 0 of entry 0: Bv4I53KnPob
parsed timeline post 1 of entry 0: Bv3mrsBHP4_
parsed timeline post 2 of entry 0: Bv1gs3wH-sw
parsed timeline post 3 of entry 0: Bv1MfooHp8P
parsed timeline post 4 of entry 0: Bv1GTOkHsVt
parsed timeline post 5 of entry 0: Bvy862BHKdW
parsed timeline post 6 of entry 0: Bvwaa8XHHH1
parsed timeline post 7 of entry 0: Bvv1KOqnAUD
parsed timeline post 8 of entry 0: BvtyxjIH16i
parsed timeline post 9 of entry 0: BvjEHW7nvR4
parsed timeline post 10 of entry 0: BvhDOaDFujX
parsed timeline post 11 of entry 0: Bvgg1TZn5qz
parsed timeline post 0 of entry 1: Bvb3cjRHEQe
parsed timeline post 1 of entry 1: BvbW6p-npze
parsed timeline post 2 of entry 1: BvUSQE_nOKt
parsed timeline post 3 of entry 1: BvTnE8Onz-v
parsed timeline post 4 of entry 1: BvRroMZnDlJ
parsed timeline post 5 of entry 1: BvRCgtBH_f-
parsed timeline post 6 of entry 1: BvO-HwfH18O
parsed timeline post 7 of entry 1: BvOdcPPnMBX
parsed timeline post 8 of entry 1: BvMe22cHL1W
parsed time

parsed timeline post 4 of entry 15: BocJQRTnrNp
parsed timeline post 5 of entry 15: BocDNHgnmjo
parsed timeline post 6 of entry 15: BobgfwsnjqW
parsed timeline post 7 of entry 15: BoQ4h8yidux
parsed timeline post 8 of entry 15: BoOY9cdHOaH
parsed timeline post 9 of entry 15: BoMR2A_HyJ2
parsed timeline post 10 of entry 15: BoJKwwWHOhr
parsed timeline post 11 of entry 15: BoHVE8JnpOJ
parsed timeline post 0 of entry 16: Bn-2N5In6Oi
parsed timeline post 1 of entry 16: Bn5rNyMgYkK
parsed timeline post 2 of entry 16: Bn3t8vWA0m3
parsed timeline post 3 of entry 16: Bn0cyUKgHFr
parsed timeline post 4 of entry 16: BntOvBOHSSt
parsed timeline post 5 of entry 16: Bnqk_zLAVuf
parsed timeline post 6 of entry 16: BnnvlfygUO2
parsed timeline post 7 of entry 16: BnluVLFnXED
parsed timeline post 8 of entry 16: BnlHD2TAW_x
parsed timeline post 9 of entry 16: BnjC2N9AcVw
parsed timeline post 10 of entry 16: BniwZefgRZU
parsed timeline post 11 of entry 16: Bnigxh3g3xk
parsed timeline post 0 of entry 17: 

parsed timeline post 6 of entry 30: BhgTrwwAMgx
parsed timeline post 7 of entry 30: Bhdl7_0gcGe
parsed timeline post 8 of entry 30: BhbfCo2gQNt
parsed timeline post 9 of entry 30: BhbCTocgp4t
parsed timeline post 10 of entry 30: BhYfEtAA6Wg
parsed timeline post 11 of entry 30: BhWlt0MgurR
parsed timeline post 0 of entry 31: BhV2oJKgLN7
parsed timeline post 1 of entry 31: BhOuyXlgUbD
parsed timeline post 2 of entry 31: BhOWbKiAUd1
parsed timeline post 3 of entry 31: BhMC7VbAlDT
parsed timeline post 4 of entry 31: BhLmrhfAP6U
parsed timeline post 5 of entry 31: BhBTAo7g8-3
parsed timeline post 6 of entry 31: Bg8ILOJAOrz
parsed timeline post 7 of entry 31: Bg5jYQIAia_
parsed timeline post 8 of entry 31: Bg0hwIvAAKQ
parsed timeline post 9 of entry 31: BgyV6eoA3DQ
parsed timeline post 10 of entry 31: Bgx9QaKgI0p
parsed timeline post 11 of entry 31: BgoCIsYg_I4
parsed timeline post 0 of entry 32: Bgn39b5AL-g
parsed timeline post 1 of entry 32: BglJzDAgilc
parsed timeline post 2 of entry 32: 

parsed timeline post 3 of entry 45: BaeOk9rgKBy
parsed timeline post 4 of entry 45: Bads2zHgbKt
parsed timeline post 5 of entry 45: BadmHv-AiW0
parsed timeline post 6 of entry 45: Bab3jzMgcSM
parsed timeline post 7 of entry 45: BabnkXmAJ8R
parsed timeline post 8 of entry 45: BabSVQSgnmB
parsed timeline post 9 of entry 45: BaYlgRggOoF
parsed timeline post 10 of entry 45: BaYc-s-APre
parsed timeline post 11 of entry 45: BaWsbXHA1kx
parsed timeline post 0 of entry 46: BaWL2j1gMaL
parsed timeline post 1 of entry 46: BaUHyLegGJ3
parsed timeline post 2 of entry 46: BaT7P_QAqk7
parsed timeline post 3 of entry 46: BaTtTpYA2Ek
parsed timeline post 4 of entry 46: BaMNZBbg75W
parsed timeline post 5 of entry 46: BaL-5xeA8yY
parsed timeline post 6 of entry 46: BaLl6o0BYhy
parsed timeline post 7 of entry 46: BaJhPl8APEK
parsed timeline post 8 of entry 46: BaHCVuRA3HM
parsed timeline post 9 of entry 46: BaGiEQIAtE8
parsed timeline post 10 of entry 46: BaD7nyBAtT5
parsed timeline post 11 of entry 46: 

parsed timeline post 5 of entry 60: BS3DcC3A5sE
parsed timeline post 6 of entry 60: BS1HtadAEDQ
parsed timeline post 7 of entry 60: BS0dvm0A-oD
parsed timeline post 8 of entry 60: BSeEyGBgowi
parsed timeline post 9 of entry 60: BSTpAjKgnIJ
parsed timeline post 10 of entry 60: BSQ9Mp9glj0
parsed timeline post 11 of entry 60: BSOcdhRgHiN
parsed timeline post 0 of entry 61: BSLZPHnAIYC
parsed timeline post 1 of entry 61: BSLDZgdALCx
parsed timeline post 2 of entry 61: BSJS4SvAa40
parsed timeline post 3 of entry 61: BSInndnALAa
parsed timeline post 4 of entry 61: BSBkd3WACsK
parsed timeline post 5 of entry 61: BSA-IDLApRl
parsed timeline post 6 of entry 61: BR_F6qVgWZR
parsed timeline post 7 of entry 61: BR-gNkxAX7v
parsed timeline post 8 of entry 61: BR8dtasABZp
parsed timeline post 9 of entry 61: BR77K1Ug6M0
parsed timeline post 10 of entry 61: BR5s7Z3Awk2
parsed timeline post 11 of entry 61: BR5Xad9gI2l
parsed timeline post 0 of entry 62: BR2xaM4APAD
parsed timeline post 1 of entry 62: 

parsed timeline post 8 of entry 74: BKVrV5Kj3us
parsed timeline post 9 of entry 74: BKTKslaD91G
parsed timeline post 10 of entry 74: BKQh44ej9ls
parsed timeline post 11 of entry 74: BKFxQFTDbxp
parsed timeline post 0 of entry 75: BJ-hjRtj4mv
parsed timeline post 1 of entry 75: BJ0br3jjpV8
parsed timeline post 2 of entry 75: BJx8FlQDCM9
parsed timeline post 3 of entry 75: BJvUeiDjQU-
parsed timeline post 4 of entry 75: BJsANO8j8j0
parsed timeline post 5 of entry 75: BJkUTzdDTS-
parsed timeline post 6 of entry 75: BJieqruD9LR
parsed timeline post 7 of entry 75: BJfHrcDjVum
parsed timeline post 8 of entry 75: BJLFGfQjyaZ
parsed timeline post 9 of entry 75: BJANkQ9DM-r
parsed timeline post 10 of entry 75: BI7yEy3j-C8
parsed timeline post 11 of entry 75: BI5Evf_j_U7
parsed timeline post 0 of entry 76: BIm4KAAjhG6
parsed timeline post 1 of entry 76: BIijIF9Dyk3
parsed timeline post 2 of entry 76: BIiNBUaDPbJ
parsed timeline post 3 of entry 76: BIhqSS9DybX
parsed timeline post 4 of entry 76: 

parsed timeline post 0 of entry 89: BBkiQwhRKDN
parsed timeline post 1 of entry 89: BBabmFsxKLy
parsed timeline post 2 of entry 89: BA-GaSpxKPk
parsed timeline post 3 of entry 89: BA2ByyzRKHg
parsed timeline post 4 of entry 89: BAuaxkQRKOC
parsed timeline post 5 of entry 89: BAoiMdYRKK-
parsed timeline post 6 of entry 89: BAm7aj5xKOj
parsed timeline post 7 of entry 89: BAmwXPyxKFi
parsed timeline post 8 of entry 89: BAmfIbNRKBI
parsed timeline post 9 of entry 89: BAkgJKgRKCE
parsed timeline post 10 of entry 89: BAkLvQXRKJb
parsed timeline post 11 of entry 89: BAjksG6xKGx
parsed timeline post 0 of entry 90: BAhwFBWxKCM
parsed timeline post 1 of entry 90: BAhinNLxKG4
parsed timeline post 2 of entry 90: BAhYEVxRKDm
parsed timeline post 3 of entry 90: BAe3necRKKK
parsed timeline post 4 of entry 90: BAcpGsIRKC8
parsed timeline post 5 of entry 90: BAcIqMsxKH4
parsed timeline post 6 of entry 90: BAaF9NPxKLt
parsed timeline post 7 of entry 90: BAFavlUxKEI
parsed timeline post 8 of entry 90: BA

parsed timeline post 7 of entry 103: 3wWIKhxKKo
parsed timeline post 8 of entry 103: 3ttfpvRKLg
parsed timeline post 9 of entry 103: 3rNytHxKFL
parsed timeline post 10 of entry 103: 3jecL0RKPj
parsed timeline post 11 of entry 103: 3gyVJlxKCd
parsed timeline post 0 of entry 104: 3elSpwRKFN
parsed timeline post 1 of entry 104: 3RUoSpxKMs
parsed timeline post 2 of entry 104: 3O3-opRKB3
parsed timeline post 3 of entry 104: 3MXevHxKMZ
parsed timeline post 4 of entry 104: 3JhbtLRKFW
parsed timeline post 5 of entry 104: 3GaYtDRKJ2
parsed timeline post 6 of entry 104: 2_QhHfRKLz
parsed timeline post 7 of entry 104: 28r5FkxKOo
parsed timeline post 8 of entry 104: 26UdJ1xKJd
parsed timeline post 9 of entry 104: 254yyhRKMH
parsed timeline post 10 of entry 104: 23pLGDxKMd
parsed timeline post 11 of entry 104: 23YV1dxKAH
parsed timeline post 0 of entry 105: 21GSvERKJl
parsed timeline post 1 of entry 105: 2tFfq9xKO-
parsed timeline post 2 of entry 105: 2qxISZxKBU
parsed timeline post 3 of entry 105:

,medium_id,updated,shortcode,owner_id,username,taken_at,typename,is_video,caption,likes,comments,video_views
0,2015399993864419867,2021-06-03 14:33:07.314,Bv4I53KnPob,38559391,mini,2019-04-05 16:27:03,GraphImage,False,Be safe when discovering new paths this summer...,25190,134,None
1,2015249486239825471,2021-06-03 14:33:07.314,Bv3mrsBHP4_,38559391,mini,2019-04-05 11:28:01,GraphImage,False,@dave_f56's MINI Hatch is perfectly poised to ...,8062,16,None
2,2014660229326695216,2021-06-03 14:33:07.314,Bv1gs3wH-sw,38559391,mini,2019-04-04 15:57:16,GraphImage,False,60 years of racing heritage going head to head...,15238,41,None
3,2014571358802845455,2021-06-03 14:33:07.314,Bv1MfooHp8P,38559391,mini,2019-04-04 13:00:42,GraphImage,False,A MINI Hatch looks just right in this winter w...,11587,30,None
4,2014544117905671533,2021-06-03 14:33:07.314,Bv1GTOkHsVt,38559391,mini,2019-04-04 12:06:34,GraphImage,False,"The world at your wheels. Thanks, @r8smv for t...",17149,46,None
...,...,...,...,...,...,...,...,...,...,...,...,...
1291,960739621803958484,2021-06-03 14:37:55.738,1VO5iHRKDU,38559391,mini,2015-04-11 12:45:09,GraphImage,False,How do you envision the future of driving? #MI...,4968,31,None
1292,960247780654490354,2021-06-03 14:37:55.738,1TfETWRKLy,38559391,mini,2015-04-10 20:27:57,GraphImage,False,Race you to the #weekend.\n#MINI #Hatch #Sunse...,8483,59,None
1293,960086242018763236,2021-06-03 14:37:55.738,1S6VmxRKHk,38559391,mini,2015-04-10 15:07:00,GraphVideo,True,Advancing safety is at the centre of our visio...,6052,450,0
1294,959938994442117829,2021-06-03 14:37:55.738,1SY23xxKLF,38559391,mini,2015-04-10 10:14:27,GraphImage,False,Turn heads when you turn a corner. MINI Augmen...,8273,82,None


In [61]:
# Check specific shortcode
df_posts.loc[df_posts['shortcode'] == 'COARxOShejL']

,medium_id,updated,shortcode,owner_id,username,taken_at,typename,is_video,caption,likes,comments,video_views


In [62]:
# Sort taken_at in descending order
df_posts.sort_values(by='taken_at',ascending=False, inplace=True)
df_posts.head(40)

,medium_id,updated,shortcode,owner_id,username,taken_at,typename,is_video,caption,likes,comments,video_views
0,2015399993864419867,2021-06-03 14:33:07.314,Bv4I53KnPob,38559391,mini,2019-04-05 16:27:03,GraphImage,False,Be safe when discovering new paths this summer...,25190,134,None
1,2015249486239825471,2021-06-03 14:33:07.314,Bv3mrsBHP4_,38559391,mini,2019-04-05 11:28:01,GraphImage,False,@dave_f56's MINI Hatch is perfectly poised to ...,8062,16,None
2,2014660229326695216,2021-06-03 14:33:07.314,Bv1gs3wH-sw,38559391,mini,2019-04-04 15:57:16,GraphImage,False,60 years of racing heritage going head to head...,15238,41,None
3,2014571358802845455,2021-06-03 14:33:07.314,Bv1MfooHp8P,38559391,mini,2019-04-04 13:00:42,GraphImage,False,A MINI Hatch looks just right in this winter w...,11587,30,None
4,2014544117905671533,2021-06-03 14:33:07.314,Bv1GTOkHsVt,38559391,mini,2019-04-04 12:06:34,GraphImage,False,"The world at your wheels. Thanks, @r8smv for t...",17149,46,None
5,2013939909909063510,2021-06-03 14:33:07.314,Bvy862BHKdW,38559391,mini,2019-04-03 16:06:30,GraphVideo,True,Get creative and #design your MINI #Cockpit Fa...,8497,74,75028
6,2013225234162545141,2021-06-03 14:33:07.314,Bvwaa8XHHH1,38559391,mini,2019-04-02 16:26:11,GraphSidecar,False,Traditionally contemporary. Discover the MINI ...,6418,14,None
7,2013061357865010435,2021-06-03 14:33:07.314,Bvv1KOqnAUD,38559391,mini,2019-04-02 11:00:36,GraphImage,False,"Roof down, no matter the weather. @ik_ki.32_20...",8798,17,None
8,2012487915801632418,2021-06-03 14:33:07.314,BvtyxjIH16i,38559391,mini,2019-04-01 16:01:16,GraphImage,False,London calling. A Classic Mini spotted amongst...,15600,48,None
9,2009467956582347896,2021-06-03 14:33:07.314,BvjEHW7nvR4,38559391,mini,2019-03-28 11:01:15,GraphVideo,True,"With a brilliant British racing green finish, ...",6602,23,31333
